# BEE 4750 Homework 4: Linear Programming and Capacity Expansion

**Name**:

**ID**:

> **Due Date**
>
> Thursday, 11/07/23, 9:00pm

## Overview

### Instructions

-   Problem 1 asks you to formulate and solve a resource allocation
    problem using linear programming.
-   Problem 2 asks you to formulate, solve, and analyze a standard
    generating capacity expansion problem.
-   Problem 3 (5750 only) asks you to add a CO<sub>2</sub> constraint to
    the capacity expansion problem and identify changes in the resulting
    solution.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [5]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `~/BEE 4750/Homework/hw4-isa-ma-bay-1`
   Installed DisplayAs ────── v0.1.6
   Installed ArgCheck ─────── v2.3.0
   Installed MarkdownTables ─ v1.1.0
Precompiling project...
  ✓ ArgCheck
  ✓ DisplayAs
  ✓ MarkdownTables
  3 dependencies successfully precompiled in 2 seconds. 188 already precompiled.


In [6]:
using JuMP
using HiGHS
using DataFrames
using Plots
using Measures
using CSV
using MarkdownTables

## Problems (Total: 50/60 Points)

### Problem 1 (20 points)

A farmer has access to a pesticide which can be used on corn, soybeans,
and wheat fields, and costs \$70/ha-yr to apply. The crop yields the
farmer can obtain following crop yields by applying varying rates of
pesticides to the field are shown in
<a href="#tbl-yields" class="quarto-xref">Table 1</a>.

| Application Rate (kg/ha) | Soybean (kg/ha) | Wheat (kg/ha) | Corn (kg/ha) |
|:------------------------:|:---------------:|:-------------:|:------------:|
|            0             |      2900       |     3500      |     5900     |
|            1             |      3800       |     4100      |     6700     |
|            2             |      4400       |     4200      |     7900     |

Table 1: Crop yields from applying varying pesticide rates for Problem
1.

The costs of production, *excluding pesticides*, for each crop, and
selling prices, are shown in
<a href="#tbl-costs" class="quarto-xref">Table 2</a>.

|   Crop   | Production Cost (\$/ha-yr) | Selling Price (\$/kg) |
|:--------:|:--------------------------:|:---------------------:|
| Soybeans |            350             |         0.36          |
|  Wheat   |            280             |         0.27          |
|   Corn   |            390             |         0.22          |

Table 2: Costs of crop production, excluding pesticides, and selling
prices for each crop.

Recently, environmental authorities have declared that farms cannot have
an *average* application rate on soybeans, wheat, and corn which exceeds
0.8, 0.7, and 0.6 kg/ha, respectively. The farmer has asked you for
advice on how they should plant crops and apply pesticides to maximize
profits over 130 total ha while remaining in regulatory compliance if
demand for each crop (which is the maximum the market would buy) this
year is 250,000 kg?

**In this problem**:

-   Formulate a linear program for this resource allocation problem,
    including clear definitions of decision variable(s) (including
    units), objective function(s), and constraint(s) (make sure to
    explain functions and constraints with any needed derivations and
    explanations). **Tip: Make sure that all of your constraints are
    linear**.
-   Implement the program in `JuMP.jl` and find the solution. How many
    ha should the farmer dedicate to each crop and with what pesticide
    application rate(s)? How much profit will the farmer expect to make?
-   The farmer has an opportunity to buy an extra 10 ha of land. How
    much extra profit would this land be worth to the farmer? Discuss
    why this value makes sense and whether you would recommend the
    farmer should make the purchase.


There are 3 crop types and for each crop type there are 3 different pesticide application possibilities creating 9 different scenarios. The decision variable is the amount of land in each scenario therefore it is the amount of land for a given crop with a given application type. We will define this as $A_{c,t}$ where c denotes the crop type and t denotes the treatment dose. Therefore we define 9 of these variables individually as follows:

soy: $A_{1,1}$, $A_{1,2}$, $A_{1,3}$

wheat: $A_{2,1}$, $A_{2,2}$, $A_{2,3}$

corn: $A_{3,1}$, $A_{3,2}$, $A_{3,3}$

The goal of the optimization is to optimize profit under the given constraints.

$profit = price_{selling} - cost_{production} - cost_{pesticide}$

To break that down further we need to write each of those terms in terms of A.

#### Selling Price:

The selling price in dollars/ha for a given crop as given in the problem will be defined as $S_c$ so the selling price for a given scenario is:

$P = [0.36, 0.27, 0.22]$

$selling price = S_c * Y_{c,t} * A_{c,t}$

#### Production Cost:

The production cost in dollars/ha-yr for a given crop as given in the problem will be defined as $C_c$ so the selling price for a given scenario is:

$ C = [350, 280, 390]$

$selling price = C_c * A_{c,t}$

#### Pesticide Cost:




In [4]:
Y = [[2900, 3500, 5900];; [3800, 4100, 6700];; [4400, 4200, 7900]]    # yield

@show Y[1,2]

# D[x,y] = xth column, yth row

Y[1, 2] = 3800


3800

In [12]:
# Define given parameters
C = [350, 280, 390]     # production cost
Cp = 70                 # pesticide cost - $/kg applied - you
# pesticide cost = 70*rate*area
P = [0.36, 0.27, 0.22]  # selling price
Y = [[2900, 3500, 5900];; [3800, 4100, 6700];; [4400, 4200, 7900]]    # yield
M = [0.8, 0.7, 0.6]     # maximum pesticide

crop_model = Model(HiGHS.Optimizer) # initialize model object
@variable(crop_model, A[1:3, 1:3] >= 0) # non-negativity constraints for 9 variables

# define objective funtion equation
#OF = sum(sum(P[x] * Y[x,y] * A[x,y] - C[x] * A[x,y] - Cp * A[x,y] for y = 1:3) for x = 1:3)
@objective(crop_model, Max, sum(sum(P[x] * Y[x,y] * A[x,y] - C[x] * A[x,y] - Cp * A[x,y] * (y-1) for y = 1:3) for x = 1:3))


# uncomment the following lines and add the objective and constraints as needed for the model
for i in 1:3 # for each crop type
        # avg pesticide use constraint
        @constraint(crop_model,  (sum(A[i,j]*(j-1) for j = 1:3)) <= M[i]* sum(A[i,j] for j=1:3))

end

@constraint(crop_model, total_area, sum(sum(A[x,y] for y in 1:3) for x in 1:3) <= 130)
@constraint(crop_model, total_yield, sum(sum(A[x,y]*Y[x,y] for y = 1:3) for x = 1:3) <= 250000)




# @constraint(crop_model, total_yield, Y[1]*A[1]+Y[2]*A[2]+Y[3]*A[3]+Y[4]*A[4]+Y[5*A[5]+Y[6]*A[6]+Y[7]*A[7]+Y[8]*A[8]+Y[9]*A[9])
# @constraint(crop_model, )

print(crop_model) # this outputs a nicely formatted summary of the model so you can check your specification

Max 694 A[1,1] + 948 A[1,2] + 1094 A[1,3] + 665.0000000000001 A[2,1] + 757 A[2,2] + 714 A[2,3] + 908 A[3,1] + 1014 A[3,2] + 1208 A[3,3]
Subject to
 -0.8 A[1,1] + 0.19999999999999996 A[1,2] + 1.2 A[1,3] ≤ 0
 -0.7 A[2,1] + 0.30000000000000004 A[2,2] + 1.3 A[2,3] ≤ 0
 -0.6 A[3,1] + 0.4 A[3,2] + 1.4 A[3,3] ≤ 0
 total_area : A[1,1] + A[2,1] + A[3,1] + A[1,2] + A[2,2] + A[3,2] + A[1,3] + A[2,3] + A[3,3] ≤ 130
 total_yield : 2900 A[1,1] + 3500 A[2,1] + 5900 A[3,1] + 3800 A[1,2] + 4100 A[2,2] + 6700 A[3,2] + 4400 A[1,3] + 4200 A[2,3] + 7900 A[3,3] ≤ 250000
 A[1,1] ≥ 0
 A[2,1] ≥ 0
 A[3,1] ≥ 0
 A[1,2] ≥ 0
 A[2,2] ≥ 0
 A[3,2] ≥ 0
 A[1,3] ≥ 0
 A[2,3] ≥ 0
 A[3,3] ≥ 0


In [13]:
optimize!(crop_model)

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [2e-01, 8e+03]
  Cost   [7e+02, 1e+03]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 2e+05]
Presolving model
5 rows, 9 cols, 27 nonzeros  0s
5 rows, 9 cols, 27 nonzeros  0s
Presolve : Reductions: rows 5(-0); columns 9(-0); elements 27(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.5167175441e+02 Ph1: 5(27.5244); Du: 9(151.672) 0s
          2     6.1961325967e+04 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 2
Objective value     :  6.1961325967e+04
HiGHS run time      :          0.00


In [15]:
@show value.(A)

value.(A) = [13.812154696132595 55.248618784530386 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]


3×3 Matrix{Float64}:
 13.8122  55.2486  0.0
  0.0      0.0     0.0
  0.0      0.0     0.0


### Problem 2 (30 points)

For this problem, we will use hourly load (demand) data from 2013 in New
York’s Zone C (which includes Ithaca). The load data is loaded and
plotted below in <a href="#fig-demand" class="quarto-xref">Figure 1</a>.

In [1]:
# load the data, pull Zone C, and reformat the DataFrame
NY_demand = DataFrame(CSV.File("data/2013_hourly_load_NY.csv"))
rename!(NY_demand, :"Time Stamp" => :Date)
demand = NY_demand[:, [:Date, :C]]
rename!(demand, :C => :Demand)
demand[:, :Hour] = 1:nrow(demand)

# plot demand
plot(demand.Hour, demand.Demand, xlabel="Hour of Year", ylabel="Demand (MWh)", label=:false)

UndefVarError: UndefVarError: `CSV` not defined

Next, we load the generator data, shown in
<a href="#tbl-generators" class="quarto-xref">Table 3</a>. This data
includes fixed costs (\$/MW installed), variable costs (\$/MWh
generated), and CO<sub>2</sub> emissions intensity (tCO<sub>2</sub>/MWh
generated).

In [2]:
gens = DataFrame(CSV.File("data/generators.csv"))

UndefVarError: UndefVarError: `CSV` not defined

Finally, we load the hourly solar and wind capacity factors, which are
plotted in <a href="#fig-cf" class="quarto-xref">Figure 2</a>. These
tell us the fraction of installed capacity which is expected to be
available in a given hour for generation (typically based on the average
meteorology).

In [3]:
# load capacify factors into a DataFrame
cap_factor = DataFrame(CSV.File("data/wind_solar_capacity_factors.csv"))

# plot January capacity factors
p1 = plot(cap_factor.Wind[1:(24*31)], label="Wind")
plot!(cap_factor.Solar[1:(24*31)], label="Solar")
xaxis!("Hour of the Month")
yaxis!("Capacity Factor")

p2 = plot(cap_factor.Wind[4344:4344+(24*31)], label="Wind")
plot!(cap_factor.Solar[4344:4344+(24*31)], label="Solar")
xaxis!("Hour of the Month")
yaxis!("Capacity Factor")

display(p1)
display(p2)

UndefVarError: UndefVarError: `CSV` not defined

You have been asked to develop a generating capacity expansion plan for
the utility in Riley County, NY, which currently has no existing
electrical generation infrastructure. The utility can build any of the
following plant types: geothermal, coal, natural gas combined cycle gas
turbine (CCGT), natural gas combustion turbine (CT), solar, and wind.

While coal, CCGT, and CT plants can generate at their full installed
capacity, geothermal plants operate at maximum 85% capacity, and solar
and wind available capacities vary by the hour depend on the expected
meteorology. The utility will also penalize any non-served demand at a
rate of \$10,000/MWh.

**In this problem**:

-   Formulate a linear program for this capacity expansion problem,
    including clear definitions of decision variable(s) (including
    units), objective function(s), and constraint(s) (make sure to
    explain functions and constraints with any needed derivations and
    explanations).
-   Implement your linear program in `JuMP.jl`. Find the optimal
    solution. How much should the utility build of each type of
    generating plant? What will the total cost be? How much energy will
    be non-served?
-   What fraction of annual generation does each plant type produce? How
    does this compare to the breakdown of built capacity that you found
    in Problem 1.5? Do these results make sense given the generator
    data?
-   Make a plot of the electricity price in each hour. Discuss any
    trends that you see.

> **Significant Digits**
>
> Use `round(x; digits=n)` to report values to the appropriate
> precision! If your number is on a different order of magnitude and you
> want to round to a certain number of significant digits, you can use
> `round(x; sigdigits=n)`.

> **Getting Variable Output Values**
>
> `value.(x)` will report the values of a `JuMP` variable `x`, but it
> will return a special container which holds other information about
> `x` that is useful for `JuMP`. This means that you can’t use this
> output directly for further calculations. To just extract the values,
> use `value.(x).data`.

> **Suppressing Model Command Output**
>
> The output of specifying model components (variable or constraints)
> can be quite large for this problem because of the number of time
> periods. If you end a cell with an `@variable` or `@constraint`
> command, I *highly* recommend suppressing output by adding a
> semi-colon after the last command, or you might find that your
> notebook crashes.

### Problem 3 (10 points)

**This problem is only required for students in BEE 5750**.

The NY state legislature is considering enacting an annual
CO<sub>2</sub> limit, which for the utility would limit the emissions in
its footprint to 1.5 MtCO<sub>2</sub>/yr.

**In this problem**:

-   Reformulate your linear program from Problem 2 with any necessary
    changes to capture the CO<sub>2</sub> limit.
-   Implement the new optimization problem and find the optimal
    solution. How much should the utility build of each type of
    generating plant? What is different from your plan from Problem 1?
    Do these changes make sense?
-   What would the value to the utility be of allowing it to emit an
    additional 1000 tCO<sub>2</sub>/yr? An additional 5000?

## References

List any external references consulted, including classmates.